In [1]:
from utils import db_tools, model_tools, data_processing
import openai
import time
from sqlalchemy import text
from mysql.connector import errorcode

# 设置Chat Model和Code Model
model_tools.current_chat_model = 'chatglm3-6b'
model_tools.current_code_model = 'codegeex2-6b'
#model_tools.load_model('chatglm3-6b')
model_use = 'chatglm_turbo'

########## 以下参数需要进行确认 ##########
save_file = f'sql_result_{model_use}.xlsx'
data_file = './data/数据表字段说明-精简1.txt'
qa_file = './qa_list-2.txt'

In [2]:
# 得到sql
def get_sql(query):
    start_time = time.time()
    sys_prompt = """我正在编写SQL，以下是数据库中的数据表和字段，请思考：哪些数据表和字段是该SQL需要的，然后编写对应的SQL，如果有多个查询语句，请尝试合并为一个。编写SQL请采用```"""
    user_prompt = f"""{table_description}
=====
我要写的SQL是：{query}
请思考：哪些数据表和字段是该SQL需要的，然后编写对应的SQL
"""
    # 调用Chat模型，model表示指定的chat模型
    if model_use in ['gpt-3.5-turbo', 'chatglm_turbo']:
        print('sys_prompt=', sys_prompt)
        print('user_prompt=', user_prompt)
        #print('model_use=', model_use)
        response = model_tools.model_chat_api(sys_prompt, user_prompt, model=model_use)
    else:
        prompt = sys_prompt + '\n=====\n' + user_prompt
        #print('prompt=', prompt)
        response = model_tools.model_chat(prompt, max_length=8000)
    return response

# 读取 数据表字段说明-精简
with open(data_file, 'r', encoding='utf-8') as file:
    table_description = file.read()
# 读取 SQL问题列表
with open(qa_file, 'r', encoding='utf-8') as file:
    qa_list = file.read()
qa_list = qa_list.split('=====')

# 保存SQL结果
sql_list = []
markdown_list = []
time_list = []
for qa in qa_list:
    query = qa
    query = query.replace('\n', '')
    print(query)
    start_time = time.time()
    # 请求生成sql
    response = get_sql(query)
    use_time = round(time.time()-start_time, 2)
    time_list.append(use_time)
    print('SQL生成时间：', use_time)
    print('response=', response)
    # 提取生成的SQL
    #sql = data_processing.get_sql_code(response, model=model_use)
    sql = data_processing.get_sql_code(response)
    print('SQL: {}'.format(sql))
    sql_list.append(sql)

获取所有客户的姓名和联系电话。
sys_prompt= 我正在编写SQL，以下是数据库中的数据表和字段，请思考：哪些数据表和字段是该SQL需要的，然后编写对应的SQL，如果有多个查询语句，请尝试合并为一个。编写SQL请采用```
user_prompt= 客户信息表（CustomerInfo）：客户ID（CustomerID）、客户姓名（Name）、性别（Gender）、出生日期（DateOfBirth）、身份证号码（IDNumber）、联系地址（Address）、联系电话（PhoneNumber）、电子邮件地址（EmailAddress）、婚姻状况（MaritalStatus）、职业（Occupation）、健康状况（HealthStatus）、客户注册日期（RegistrationDate）、客户类型（CustomerType）、客户来源（SourceOfCustomer）、客户状态（CustomerStatus）

保单信息表（PolicyInfo）：保单号（PolicyNumber）、客户ID（CustomerID）、保险产品ID（ProductID）、保单状态（PolicyStatus）、受益人（Beneficiary）、受益人关系（Relationship）、投保日期（PolicyStartDate）、终止日期（PolicyEndDate）、保费支付状态（PremiumPaymentStatus）、保费支付日期（PaymentDate）、保费支付方式（PaymentMethod）、代理人ID（AgentID）

理赔信息表（ClaimInfo）：理赔号（ClaimNumber）、保单号（PolicyNumber）、理赔日期（ClaimDate）、理赔类型（ClaimType）、理赔金额（ClaimAmount）、理赔状态（ClaimStatus）、理赔描述（ClaimDescription）、受益人ID（BeneficiaryID）、医疗记录（MedicalRecords）、事故报告（AccidentReport）、审核人（ClaimHandler）、审核日期（ReviewDate）、支付方式（PaymentMethod）、支付日期（PaymentDate）、拒赔原因（DenialReason）

受益人信息表（BeneficiaryInfo）：受益人I

In [3]:
import pandas as pd

result = pd.DataFrame(columns=['QA', 'SQL', 'time'])
result['QA'] = qa_list
result['SQL'] = sql_list
result['time'] = time_list
result.to_excel(save_file, index=False)
result

,QA,SQL,time
0,获取所有客户的姓名和联系电话。\n,"SELECT Name, PhoneNumber\nFROM CustomerInfo;\n",2.58
1,\n找出所有已婚客户的客户ID和配偶姓名。\n,"SELECT c.CustomerID, b.Name AS SpouseName\nFRO...",9.03
2,\n查询所有未支付保费的保单号和客户姓名。\n,"SELECT p.PolicyNumber, c.Name\nFROM PolicyInfo...",5.44
3,\n找出所有理赔金额大于10000元的理赔记录，并列出相关客户的姓名和联系电话。\n,"SELECT \n c.Name AS CustomerName,\n c.Ph...",9.41
4,\n查找代理人的姓名和执照到期日期，按照执照到期日期升序排序。\n,"SELECT Name, LicenseExpirationDate\nFROM Agent...",2.61
5,\n获取所有保险产品的产品名称和保费，按照保费降序排序。\n,"SELECT ProductName, Premium\nFROM ProductInfo\...",2.92
6,\n查询销售区域在上海的员工，列出他们的姓名和职位。\n,"SELECT Name, Position\nFROM EmployeeInfo\nWHER...",3.63
7,\n找出所有年龄在30岁以下的客户，并列出其客户ID、姓名和出生日期。\n,"SELECT CustomerID, Name, DateOfBirth\nFROM Cus...",4.68
8,\n查找所有已审核但尚未支付的理赔记录，包括理赔号、审核人和审核日期。\n,"SELECT\n ClaimNumber,\n ClaimHandler,\n ...",7.53
9,\n获取每个产品类型下的平均保费，以及该产品类型下的产品数量。,"SELECT \n ProductType,\n AVG(Premium) AS...",4.26
